In [1]:
import numpy as np
import pandas as pd
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from textblob import TextBlob
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
import joblib




In [2]:
df_hate_1 = pd.read_csv("Hate_Speeches_1.csv")
df_hate_2 = pd.read_csv("Hate_Speeches_2.csv")
df_hate_3 = pd.read_csv("Hate_Speeches_3.csv")
df_hate_4 = pd.read_csv("Hate_Speeches_4.csv")
df_normal_5 = pd.read_csv("Normal_Speeches_1.csv")
df_normal_6 = pd.read_csv("Normal_Speeches_2.csv")
df_normal_7 = pd.read_csv("Normal_Speeches_3.csv")
df_normal_8 = pd.read_csv("Normal_Speeches_4.csv")
# df_offensive_9 = pd.read_csv("Offensive_Speeches_1.csv")
# df_offensive_10 = pd.read_csv("Offensive_Speeches_2.csv")
# df_offensive_11 = pd.read_csv("Offensive_Speeches_3.csv")
# df_offensive_12 = pd.read_csv("Offensive_Speeches_4.csv")

In [3]:

df_hate_1['speech'] = 'hate_or_offensive'
df_hate_2['speech'] = 'hate_or_offensive'
df_hate_3['speech'] = 'hate_or_offensive'
df_hate_4['speech'] = 'hate_or_offensive'

df_normal_5['speech'] = 'normal'
df_normal_6['speech'] = 'normal'
df_normal_7['speech'] = 'normal'
df_normal_8['speech'] = 'normal'

# df_offensive_9['speech'] = 'hate_or_offensive'
# df_offensive_10['speech'] = 'hate_or_offensive'
# df_offensive_11['speech'] = 'hate_or_offensive'
# df_offensive_12['speech'] = 'hate_or_offensive'

In [4]:
# frames = [df_hate_1, df_hate_2, df_hate_3, df_hate_4, df_normal_5, df_normal_6, df_normal_7, df_normal_8, df_offensive_9, df_offensive_10, df_offensive_11, df_offensive_12]
# df = pd.concat(frames)
frames = [df_hate_1, df_hate_2, df_hate_3, df_hate_4, df_normal_5, df_normal_6, df_normal_7, df_normal_8]
df = pd.concat(frames)

In [5]:
df.head(10)

,conversation_id,created_at,favorite_count,full_text,hashtags/0,hashtags/1,hashtags/2,hashtags/3,hashtags/4,hashtags/5,...,user_mentions/43/screen_name,user_mentions/44/id_str,user_mentions/44/name,user_mentions/44/screen_name,user_mentions/45/id_str,user_mentions/45/name,user_mentions/45/screen_name,user_mentions/46/id_str,user_mentions/46/name,user_mentions/46/screen_name
0,1624652050448674816,2023-02-12T06:10:08.000Z,115,another day to hate some kt3umes. they were ne...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1624649867720007680,2023-02-12T06:01:27.000Z,6,#SMG4's:\n Twitter comments vs ...,SMG4,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1624647737039007750,2023-02-12T05:52:59.000Z,1474,I hate it whenever I see “mataba ka!” comments...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1624560613807607808,2023-02-12T00:06:47.000Z,30,ain’t no way they deleted my comments under th...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1624540978047553536,2023-02-11T22:48:46.000Z,90,Just incase any of you wanna add this one to y...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1624490492644794370,2023-02-11T19:28:09.000Z,0,Honestly done and disappointed with @GenesysGo...,solana,shadowdrive,genesysgo,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1624218947242565633,2023-02-11T01:29:08.000Z,384,He's A Psycho! - Why David Goggins Records Him...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1624214848451100677,2023-02-11T01:12:50.000Z,288,Comments are fucking horrid on TikTok I hate m...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1624156000369446912,2023-02-10T21:19:00.000Z,15939,HELLO??? ALL THE CONCEPT SKETCHES FOR FREE??? ...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1623653902523441156,2023-02-09T12:03:51.000Z,1463,I love how much these pics trigger ppl. Going ...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df.columns.tolist()

['conversation_id',
 'created_at',
 'favorite_count',
 'full_text',
 'hashtags/0',
 'hashtags/1',
 'hashtags/2',
 'hashtags/3',
 'hashtags/4',
 'hashtags/5',
 'hashtags/6',
 'hashtags/7',
 'hashtags/8',
 'hashtags/9',
 'hashtags/10',
 'id',
 'media/0/media_url',
 'media/0/type',
 'media/1/media_url',
 'media/1/type',
 'media/2/media_url',
 'media/2/type',
 'media/3/media_url',
 'media/3/type',
 'reply_count',
 'retweet_count',
 'startUrl',
 'symbols/0',
 'url',
 'urls/0/display_url',
 'urls/0/expanded_url',
 'urls/0/url',
 'urls/1/display_url',
 'urls/1/expanded_url',
 'urls/1/url',
 'user/advertiser_account_type',
 'user/blocked_by',
 'user/blocking',
 'user/business_profile_state',
 'user/can_dm',
 'user/contributors_enabled',
 'user/created_at',
 'user/default_profile_image',
 'user/description',
 'user/ext_has_nft_avatar',
 'user/ext_is_blue_verified',
 'user/ext_verified_type',
 'user/fast_followers_count',
 'user/favourites_count',
 'user/follow_request_sent',
 'user/followed_by'

In [7]:
df.insert(4, 'speech', df.pop('speech'))

<ipython-input-7-5bb44bd9e284>:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.insert(4, 'speech', df.pop('speech'))


In [8]:
df.head(10)

,conversation_id,created_at,favorite_count,full_text,speech,hashtags/0,hashtags/1,hashtags/2,hashtags/3,hashtags/4,...,user_mentions/43/screen_name,user_mentions/44/id_str,user_mentions/44/name,user_mentions/44/screen_name,user_mentions/45/id_str,user_mentions/45/name,user_mentions/45/screen_name,user_mentions/46/id_str,user_mentions/46/name,user_mentions/46/screen_name
0,1624652050448674816,2023-02-12T06:10:08.000Z,115,another day to hate some kt3umes. they were ne...,hate_or_offensive,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1624649867720007680,2023-02-12T06:01:27.000Z,6,#SMG4's:\n Twitter comments vs ...,hate_or_offensive,SMG4,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1624647737039007750,2023-02-12T05:52:59.000Z,1474,I hate it whenever I see “mataba ka!” comments...,hate_or_offensive,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1624560613807607808,2023-02-12T00:06:47.000Z,30,ain’t no way they deleted my comments under th...,hate_or_offensive,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1624540978047553536,2023-02-11T22:48:46.000Z,90,Just incase any of you wanna add this one to y...,hate_or_offensive,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1624490492644794370,2023-02-11T19:28:09.000Z,0,Honestly done and disappointed with @GenesysGo...,hate_or_offensive,solana,shadowdrive,genesysgo,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1624218947242565633,2023-02-11T01:29:08.000Z,384,He's A Psycho! - Why David Goggins Records Him...,hate_or_offensive,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1624214848451100677,2023-02-11T01:12:50.000Z,288,Comments are fucking horrid on TikTok I hate m...,hate_or_offensive,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1624156000369446912,2023-02-10T21:19:00.000Z,15939,HELLO??? ALL THE CONCEPT SKETCHES FOR FREE??? ...,hate_or_offensive,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1623653902523441156,2023-02-09T12:03:51.000Z,1463,I love how much these pics trigger ppl. Going ...,hate_or_offensive,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
df = df[['id','full_text', 'speech' ]]


In [10]:
df.head(10)

,id,full_text,speech
0,1624652050448674816,another day to hate some kt3umes. they were ne...,hate_or_offensive
1,1624649867720007680,#SMG4's:\n Twitter comments vs ...,hate_or_offensive
2,1624647737039007750,I hate it whenever I see “mataba ka!” comments...,hate_or_offensive
3,1624560613807607808,ain’t no way they deleted my comments under th...,hate_or_offensive
4,1624540978047553536,Just incase any of you wanna add this one to y...,hate_or_offensive
5,1624490492644794370,Honestly done and disappointed with @GenesysGo...,hate_or_offensive
6,1624218947242565633,He's A Psycho! - Why David Goggins Records Him...,hate_or_offensive
7,1624214848451100677,Comments are fucking horrid on TikTok I hate m...,hate_or_offensive
8,1624156000369446912,HELLO??? ALL THE CONCEPT SKETCHES FOR FREE??? ...,hate_or_offensive
9,1623653902523441156,I love how much these pics trigger ppl. Going ...,hate_or_offensive


In [11]:
df.loc[df['speech'] == 'normal']



,id,full_text,speech
0,1624741969045123072,"Freedom is poetry, taking liberties with words...",normal
1,1624703783115825155,"How is going,Biu?Are you passing peacefully an...",normal
2,1624745424224411649,u got ball sniffer on your team ofc u would fi...,normal
3,1624684918100971521,The sensible normal British public have had en...,normal
4,1624659992908382208,There is a conspiracy to declare all concerned...,normal
...,...,...,...
295,1622461473703215105,My experience online has suggested to me that ...,normal
296,1622331473440260098,Bro want the biggest most political arc in rec...,normal
297,1622323014971826178,It's comedic how HxH stans only defenses are t...,normal
298,1622082249402335234,i have a stutter. you don't really notice it f...,normal


In [12]:
df.loc[df['speech'] == 'hate_or_offensive']


,id,full_text,speech
0,1624652050448674816,another day to hate some kt3umes. they were ne...,hate_or_offensive
1,1624649867720007680,#SMG4's:\n Twitter comments vs ...,hate_or_offensive
2,1624647737039007750,I hate it whenever I see “mataba ka!” comments...,hate_or_offensive
3,1624560613807607808,ain’t no way they deleted my comments under th...,hate_or_offensive
4,1624540978047553536,Just incase any of you wanna add this one to y...,hate_or_offensive
...,...,...,...
217,1622626029658427393,I hate people who only text a couple words at ...,hate_or_offensive
218,1622004819811606528,Yemi makes me hate the words “I got you” If a ...,hate_or_offensive
219,1621997313123094528,In other words... they hate the truth being to...,hate_or_offensive
220,1621886030998814722,"In Rajasthan's Barmer, Swami Ramdev was very c...",hate_or_offensive


# BORDER

In [13]:
df_reddit = pd.read_csv('Reddit_Data.csv')
df_tweet = pd.read_csv('Twitter_Data.csv')

In [14]:
df_reddit.head()

,clean_comment,category
0,family mormon have never tried explain them t...,1
1,buddhism has very much lot compatible with chr...,1
2,seriously don say thing first all they won get...,-1
3,what you have learned yours and only yours wha...,0
4,for your own benefit you may want read living ...,1


In [15]:
len(df_reddit[df_reddit['category'] == 1])


15830

In [16]:

len(df_reddit[df_reddit['category'] == -1])


8277

In [17]:
df_reddit[df_reddit['category'] == 1].head(30)


,clean_comment,category
0,family mormon have never tried explain them t...,1
1,buddhism has very much lot compatible with chr...,1
4,for your own benefit you may want read living ...,1
6,was teens when discovered zen meditation was ...,1
9,dont worry about trying explain yourself just ...,1
10,recently told family that buddhist live the b...,1
11,unto others you would have them unto you woul...,1
12,first understand that you are not anyway contr...,1
13,recently heard similar question where person ...,1
14,different times different cultures same point ...,1


In [18]:
df_reddit = df_reddit[df_reddit['category'] == 1]


In [19]:
df_reddit['speech'] = 'normal'

In [20]:
df_reddit = df_reddit.rename(columns={'clean_comment': 'full_text'})


In [21]:
df_reddit.head()

,full_text,category,speech
0,family mormon have never tried explain them t...,1,normal
1,buddhism has very much lot compatible with chr...,1,normal
4,for your own benefit you may want read living ...,1,normal
6,was teens when discovered zen meditation was ...,1,normal
9,dont worry about trying explain yourself just ...,1,normal


In [22]:
df_reddit = df_reddit.drop('category', axis=1)


In [23]:
df_reddit.head()

,full_text,speech
0,family mormon have never tried explain them t...,normal
1,buddhism has very much lot compatible with chr...,normal
4,for your own benefit you may want read living ...,normal
6,was teens when discovered zen meditation was ...,normal
9,dont worry about trying explain yourself just ...,normal


In [24]:
len(df_reddit[df_reddit['speech'] == 'normal'])


15830

In [25]:
df_hate = pd.read_csv('hate_speech_dataset.csv')

In [26]:
df_hate.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [27]:
df_hate.columns.tolist()

['Unnamed: 0',
 'count',
 'hate_speech',
 'offensive_language',
 'neither',
 'class',
 'tweet']

In [28]:
df_hate = df_hate[['tweet','hate_speech', 'offensive_language', 'neither', 'class']]


In [29]:
df_hate.head(30)

,tweet,hate_speech,offensive_language,neither,class
0,!!! RT @mayasolovely: As a woman you shouldn't...,0,0,3,2
1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,0,3,0,1
2,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,0,3,0,1
3,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,0,2,1,1
4,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,0,6,0,1
5,"!!!!!!!!!!!!!!!!!!""@T_Madison_x: The shit just...",1,2,0,1
6,"!!!!!!""@__BrighterDays: I can not just sit up ...",0,3,0,1
7,!!!!&#8220;@selfiequeenbri: cause I'm tired of...,0,3,0,1
8,""" &amp; you might not get ya bitch back &amp; ...",0,3,0,1
9,""" @rhythmixx_ :hobbies include: fighting Maria...",1,2,0,1


In [30]:
df_hate.tail(30)

,tweet,hate_speech,offensive_language,neither,class
24753,you ain't gotta be a dyke to like hoes,1,2,0,1
24754,"you are a hoe, hoe, &amp; a hoe.",0,3,0,1
24755,you bitches love yall some corny nigga,0,3,0,1
24756,you can masturbate anytime bitch lol &#8220;@g...,0,3,0,1
24757,you can never get a group of hoes together wit...,0,3,0,1
24758,you can tell when dick recently been in a puss...,0,3,0,1
24759,you can't cuff a hoe lmao,0,3,0,1
24760,you drove me redneck crazy,0,2,1,1
24761,you fake niggah lolol,0,3,0,1
24762,"you got niggas, and i got bitches.",1,2,0,1


In [31]:
df_hate[df_hate['class'] == 2].head(30)


,tweet,hate_speech,offensive_language,neither,class
0,!!! RT @mayasolovely: As a woman you shouldn't...,0,0,3,2
40,""" momma said no pussy cats inside my doghouse """,0,1,2,2
63,"""@Addicted2Guys: -SimplyAddictedToGuys http://...",0,0,3,2
66,"""@AllAboutManFeet: http://t.co/3gzUpfuMev"" woo...",0,1,2,2
67,"""@Allyhaaaaa: Lemmie eat a Oreo &amp; do these...",0,1,2,2
70,"""@ArizonasFinest6: Why the eggplant emoji doe?...",0,0,3,2
75,"""@BabyAnimalPics: baby monkey bathtime http://...",0,1,2,2
115,"""@DomWorldPeace: Baseball season for the win. ...",0,0,3,2
118,"""@DunderbaIl: I'm an early bird and I'm a nigh...",0,0,3,2
119,"""@EdgarPixar: Overdosing on heavy drugs doesn'...",0,1,2,2


In [32]:
df_normal = df_hate[df_hate['class'] == 1]
df_hate = df_hate[df_hate['class'] == 1]


In [33]:
df_hate = df_hate.rename(columns={'tweet': 'full_text'})
df_hate['speech'] = 'hate_or_offensive'
df_hate = df_hate[['full_text', 'speech']]
df_normal = df_normal.rename(columns={'tweet': 'full_text'})
df_normal['speech'] = 'normal'
df_normal = df_normal[['full_text', 'speech']]

In [34]:
frames = [df, df_reddit, df_hate, df_normal]
df = pd.concat(frames)

# BORDER

In [35]:
# Download necessary NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [36]:
# Lowercase the text
df['full_text'] = df['full_text'].apply(lambda x: x.lower())

In [37]:
# Tokenize the text
df['full_text'] = df['full_text'].apply(lambda x: nltk.word_tokenize(x))

In [38]:
df.head(10)

,id,full_text,speech
0,1.624652e+18,"[another, day, to, hate, some, kt3umes, ., the...",hate_or_offensive
1,1.624650e+18,"[#, smg4, 's, :, twitter, comments, vs, youtub...",hate_or_offensive
2,1.624648e+18,"[i, hate, it, whenever, i, see, “, mataba, ka,...",hate_or_offensive
3,1.624561e+18,"[ain, ’, t, no, way, they, deleted, my, commen...",hate_or_offensive
4,1.624541e+18,"[just, incase, any, of, you, wan, na, add, thi...",hate_or_offensive
5,1.624490e+18,"[honestly, done, and, disappointed, with, @, g...",hate_or_offensive
6,1.624219e+18,"[he, 's, a, psycho, !, -, why, david, goggins,...",hate_or_offensive
7,1.624215e+18,"[comments, are, fucking, horrid, on, tiktok, i...",hate_or_offensive
8,1.624156e+18,"[hello, ?, ?, ?, all, the, concept, sketches, ...",hate_or_offensive
9,1.623654e+18,"[i, love, how, much, these, pics, trigger, ppl...",hate_or_offensive


In [39]:
# Remove stopwords
stop_words = set(stopwords.words('english'))
df['full_text'] = df['full_text'].apply(lambda x: [word for word in x if word not in stop_words])


In [40]:
# Perform lemmatization
# lemmatizer = WordNetLemmatizer()
# df['full_text'] = df['full_text'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])


In [41]:

# Remove punctuations and special characters
df['full_text'] = df['full_text'].apply(lambda x: [word for word in x if word.isalnum()])


In [42]:
# Spell correction
# df['full_text'] = df['full_text'].apply(lambda x: [str(TextBlob(word).correct()) for word in x])


In [43]:
# Create a feature matrix and target vector
X = df['full_text']
y = df['speech']

In [44]:

X = [' '.join(x) for x in X]

In [45]:
# Vectorize the text data
vectorizer = CountVectorizer(stop_words='english')
X = vectorizer.fit_transform(X)

In [46]:
print(X)

  (0, 12617)	1
  (0, 20876)	1
  (0, 26498)	1
  (0, 19973)	1
  (0, 23787)	1
  (0, 7536)	1
  (0, 36465)	1
  (0, 17993)	1
  (0, 46621)	1
  (0, 8256)	1
  (0, 28659)	1
  (0, 20881)	1
  (0, 10581)	1
  (0, 29595)	1
  (0, 41172)	1
  (0, 24291)	1
  (0, 19373)	1
  (0, 47439)	1
  (0, 27500)	1
  (1, 20876)	1
  (1, 10581)	2
  (1, 42787)	1
  (1, 48009)	1
  (1, 49955)	2
  (1, 51820)	1
  :	:
  (56068, 47395)	1
  (56068, 40928)	1
  (56068, 1834)	1
  (56068, 1835)	1
  (56068, 1837)	1
  (56068, 27484)	1
  (56068, 31231)	1
  (56068, 22230)	1
  (56069, 27534)	1
  (56069, 50128)	1
  (56069, 31353)	1
  (56069, 51777)	1
  (56069, 12546)	1
  (56069, 15382)	1
  (56069, 14037)	1
  (56069, 18574)	1
  (56069, 8021)	1
  (56069, 2949)	1
  (56069, 32210)	1
  (56070, 6795)	1
  (56070, 19799)	1
  (56070, 27476)	1
  (56070, 50843)	1
  (56070, 45869)	1
  (56070, 51825)	1


In [47]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [48]:
# Train a Naive Bayes classifier
nb = MultinomialNB()
nb.fit(X_train, y_train)

MultinomialNB()

In [49]:
# Make predictions on the test set
y_pred = nb.predict(X_test)

In [50]:
# Evaluate the model performance
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.6290571870170015


In [51]:
# # Preprocess the input text
# input_text = "hello"
# input_text = input_text.lower()
# input_text = nltk.word_tokenize(input_text)
# input_text = [word for word in input_text if word not in stop_words]
# input_text = [lemmatizer.lemmatize(word) for word in input_text]
# input_text = [word for word in input_text if word.isalnum()]
# input_text = [' '.join(input_text)]

# # Vectorize the input text data
# input_text = vectorizer.transform(input_text)

# # Use the trained model to predict the label of the input text data
# y_pred = nb.predict(input_text)

# print(y_pred)

In [68]:
# Preprocess the custom input text
custom_input = "you taste good"
custom_input = custom_input.lower()
custom_input = nltk.word_tokenize(custom_input)
# custom_input = [word for word in custom_input if word not in stop_words]
# custom_input = [lemmatizer.lemmatize(word) for word in custom_input]
custom_input = [word for word in custom_input if word.isalnum()]
custom_input = [' '.join(custom_input)]

# Vectorize the custom input text
custom_input = vectorizer.transform(custom_input)

# Make a prediction on the custom input text
prediction = nb.predict(custom_input)

# Print the prediction
print(prediction)

['normal']


In [69]:
len(df[df['speech'] == 'hate_or_offensive'])


20156